# Aqui inicia mi sufrimiento :(

# PROYECTO FINAL


## Sistemas Distribuidos con PySpark

Este proyecto implementa lo que se ha visto en clases en una serie de ejercicios a las cuales llamo un **sistema de recomendación de libros** utilizando:
- **100 libros** más descargados de Project Gutenberg (Omitiendo 2 ya que estos denegaban el acceso...)
- **PySpark** para procesamiento distribuido
- **TF-IDF** para análisis de texto
- **Similitud coseno** para encontrar libros similares
- Y las enseñanzas del profe

## CELDA 1: Configuración de Rutas

**¿Qué hace?**
Configura las rutas del proyecto para que Python pueda encontrar nuestros archivos de utilidades (`src/utils.py`).

**Explicación técnica:**
- `sys.path.append()` añade directorios donde Python buscará módulos <details> <summary> ¿Por que? </summary> (Antes tuvimos problemas con que el programa que no encontraba la ruta de SRC)</details>


- `..` = directorio padre (raíz del proyecto)
- `../src` = carpeta de código fuente<details>
    <summary> ¿Que hay ahi? </summary> 
    Ahi adentro se encuentran los archivos python donde uno descarga los libros (download_books.py) y otro donde se encarga de limpiar estos mismos como quitar los headers o footers para que no extorben en el analisis (utils.py)
</details>


In [1]:
import sys
import os

# Agregar rutas del proyecto
sys.path.append(os.path.abspath(".."))  # raíz del proyecto
sys.path.append(os.path.abspath("../src"))  # carpeta src

# Verificar rutas
print("Rutas configuradas:")
for path in sys.path[-3:]:
    print(f"  - {path}")

Rutas configuradas:
  - /home/arturoallen/proyecto_52_sistemas/lib/python3.12/site-packages
  - /home/arturoallen/Descargas/proyecto_sistemas-main
  - /home/arturoallen/Descargas/proyecto_sistemas-main/src


## CELDA 2: Descargar Recursos de NLTK

**¿Qué hace?**
Descarga las herramientas necesarias de NLTK para procesar texto en inglés.

**Lo que descarga:**
- `punkt` y `punkt_tab` → Separa texto en palabras (tokenización)<details><summary>En otras palabras</summary>Esto divide un texto grande en piezas más pequeñas llamadas tokens.</details>

- `stopwords` → Lista de palabras comunes que no aportan significado ("the", "a", "is") <details>
  <summary> ¿De donde sacamos esta herramienta? </summary>
  Esta herramienta o mas bien libreria fue recomendada por el profesor para facilitarnos la detecciòn de stopwords
</details>

In [2]:
import nltk

print("Descargando recursos de NLTK...")
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("✓ NLTK configurado")

Descargando recursos de NLTK...
✓ NLTK configurado


## CELDA 3: Inicializar Apache Spark

**¿Qué hace?**
Arranca el motor de Spark que procesará nuestros datos de forma distribuida.

**Configuración:**
- `driver.memory: 6g` → Memoria del coordinador (6 GB)<details><summary>Mas detalles</summary>Memoria para el driver.
    - Este driver.memory recibe las instrucciones, reparte el trabajo y al final recoge los resultados.
    - Los 6 GB significa que la máquina virtual va a reservar 6 GB solo para este coordinador.
    - Esto procesara los 100 libros
</details>

- `executor.memory: 4g` → Memoria de los trabajadores (4 GB)
- `local[*]` → Usa todos los cores de tu CPU

\
**¿Por qué Spark?**
- Para procesar 100 libros (~75 MB) de forma eficiente y paralela.<details>
    <summary> Y... </summary> 
    ...porque el profe nos lo pidio
</details>


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProyectoFinal_RecomendacionLibros") \
    .master("local[*]") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Reducir logs
spark.sparkContext.setLogLevel("WARN")

print("✓ Spark inicializado")
print(f"  Version: {spark.version}")
print(f"  Master: {spark.sparkContext.master}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/08 22:40:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/08 22:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✓ Spark inicializado
  Version: 4.0.1
  Master: local[*]


## CELDA 4: Importar Funciones de Utilidad



**¿Qué hace?**\
Carga las funciones que creamos para procesar los libros desde la carpeta src al archivo utils.py

**Funciones importadas:**
- `read_txt()` → Lee un archivo .txt sin importar el tipo de encoding, evitando errores por acentos o caracteres raros.
- `strip_gutenberg_headers()` → Quita las licencias, advertencias y texto extra que agregan los libros.
- `preprocess_text()` → Limpia el texto aplicando estos filtros:
    - minúsculas
    - tokenizar
    - quitar stopwords
    - quitar puntuación<br>
- `load_all_books()` → Busca todos los .txt de la carpeta y:\
  Es la función que carga todos los libros del proyecto.
    - los lee
    - los limpia
    - los convierte en una lista/diccionario de libros procesados

In [4]:
from src.utils import (
    read_txt,
    strip_gutenberg_headers,
    preprocess_text,
    load_all_books
)

print("✓ Utilidades importadas")

✓ Utilidades importadas


## CELDA 5 y 6: Cargar 100 Libros

**¿Qué hace?**
Lee los 100 lirbos .txt desde la carpeta `data/` y los preprocesa.

**¿En que consiste el proceso?**
1. Lee el archivo
2. Elimina headers/footers
3. Convierte a minúsculas
4. Separa en palabras (tokens)
5. Elimina stopwords ("the", "a", etc.)
6. Elimina puntuación y números

**Resultado:**
Una lista con 100 libros, cada uno con:
- ID del libro (usando los ID originales de la pagina)
- Nombre del archivo
- Texto completo
- Lista de palabras limpias (tokens)

In [5]:
# Desmarcar el comentario en caso de extrema emergencia (No tener los libros)
# python3 src/download_books.py

In [6]:
data_dir = "../data"
books = load_all_books(data_dir, max_books=100)

print(f"\n📊 Resumen de carga:")
print(f"   Total de libros: {len(books)}")
if books:
    ejemplo = books[0]
    print(f"   Ejemplo - ID: {ejemplo[0]}, Tokens: {len(ejemplo[3])}")


📚 Cargando 7 libros desde ../data/
✅ Total de libros cargados exitosamente: 7


📊 Resumen de carga:
   Total de libros: 7
   Ejemplo - ID: If the Eternity Should Fail, Tokens: 206


## CELDA 7: Crear DataFrame de Spark


**¿Qué hace?**
Convierte nuestra lista de libros en un **DataFrame de Spark** (como una especie de tabla de Excel gigante).

**Columnas del DataFrame:**
- `book_id` → ID único del libro (ej: "84", "1342")
- `title` → Nombre del archivo (ej: "84.txt")
- `text` → Texto completo del libro
- `tokens` → Lista de palabras procesadas

**¿Por qué un DataFrame?**
Spark puede procesar DataFrames de forma paralela y distribuida.
<details><summary>Comentario</summary> Creemos que no se ve las columnas de "text" y "tokens" porque estamos usando una maquina virutal para correr este codigo y la verdad a este punto en el que estoy hubiera preferido instalar linux pero en la laptop, casi no la uso</details>
<br/>


In [7]:
from pyspark.sql.types import ArrayType, StringType
print("Creando DataFrame de Spark...")
df = spark.createDataFrame(
    [(b[0], b[1], b[3]) for b in books],
    schema=["book_id", "title", "tokens"]
)
print(f"✓ DataFrame creado con {df.count()} documentos\n")
df.select("book_id", "title").show(5, truncate=False)

Creando DataFrame de Spark...


✓ DataFrame creado con 7 documentos

+---------------------------+-------------------------------+
|book_id                    |title                          |
+---------------------------+-------------------------------+
|If the Eternity Should Fail|If the Eternity Should Fail.txt|
|Man On The Edge            |Man On The Edge.txt            |
|Speed Of Light             |Speed Of Light.txt             |
|The Great Unknown          |The Great Unknown.txt          |
|The Red And The Black      |The Red And The Black.txt      |
+---------------------------+-------------------------------+
only showing top 5 rows


## CELDA 8: Crear Vocabulario (CountVectorizer)

**¿Qué hace?**\
Crea un **vocabulario** con las 2000 palabras más importantes de todos los libros.

**Proceso:**
1. Cuenta cuántas veces aparece cada palabra en cada libro
2. Selecciona las 2000 palabras más frecuentes
3. Filtra palabras que aparecen en menos de 2 libros (muy raras)

**Resultado:**\
Cada libro se representa como un vector de 2000 números (frecuencias de palabras).
<details>
  <summary>Detalles del codigo</summary>
  <p>Parámetros:</p>
  <ul>
    <li>VOCAB_SIZE=2000: limita el vocabulario a las 2000 palabras más frecuentes. Evita vectores enormes.</li>
    <li>MIN_DF=2: elimina palabras que aparecen en menos de 2 documentos (filtra ruido / typos).</li>
  </ul>
</details>





In [8]:
from pyspark.ml.feature import CountVectorizer

print("Creando vocabulario con CountVectorizer...")

VOCAB_SIZE = 2000
MIN_DF = 2

cv = CountVectorizer(
    inputCol="tokens",
    outputCol="raw_features",
    vocabSize=VOCAB_SIZE,
    minDF=MIN_DF
)

cv_model = cv.fit(df)
df = cv_model.transform(df)
df = df.drop("tokens")  

actual_vocab_size = len(cv_model.vocabulary)
print(f"✓ Vocabulario creado")
print(f"  Tamaño del vocabulario: {actual_vocab_size} palabras")
print(f"  Top 10 palabras: {cv_model.vocabulary[:10]}")

Creando vocabulario con CountVectorizer...


✓ Vocabulario creado
  Tamaño del vocabulario: 93 palabras
  Top 10 palabras: ['falling', 'eternity', 'time', 'line', 'world', 'nothing', 'edge', 'waiting', 'us', 'ending']


## CELDA 9: Calcular TF-IDF

**¿Qué hace?**\
Calcula **TF-IDF** (Term Frequency - Inverse Document Frequency) para cada palabra.

**¿Qué es TF-IDF?**\
Un número que indica el **peso** o **importancia** en una palabra para un libro específico.

**Fórmula simple:**
- **TF** (frecuencia): ¿Cuántas veces aparece en ESTE libro?
- **IDF** (rareza): ¿Qué tan rara es en TODOS los libros?
- **TF-IDF** = TF × IDF

**Ejemplo:**
- "elizabeth" aparece mucho en Frankenstein → TF alto
- "elizabeth" NO aparece en otros libros → IDF alto
- **TF-IDF de "elizabeth"** = ALTO (palabra característica)

**¿Al final que se obtiene de todo esto?:**\
Cada libro tiene un vector TF-IDF que representa su contenido único.


In [9]:
from pyspark.ml.feature import IDF

print("\nCalculando TF-IDF...")

idf = IDF(inputCol="raw_features", outputCol="tfidf")
idf_model = idf.fit(df)
df = idf_model.transform(df)
df = df.drop("raw_features") 

print("✓ TF-IDF calculado")


Calculando TF-IDF...


[Stage 10:=======>                                                  (1 + 7) / 8]

✓ TF-IDF calculado


## CELDA 10: Normalizar Vectores


**¿Qué hace?**\
Normaliza los vectores TF-IDF para que todos tengan la misma "longitud" matemática.<details> <summary>Ejemplo</summary> Es como comparar la composición de dos bebidas sin importar el tamaño del vaso.</details>

**¿Por qué normalizar?**
- Libros largos tienen números más grandes
- Queremos comparar **proporcionalmente**, no por tamaño
- Después de normalizar, los valores están entre 0 y 1


**Resultado:**\
Vectores `tfidf_norm` con valores entre 0 y 1.


In [10]:
from pyspark.ml.feature import Normalizer

print("Normalizando vectores TF-IDF...")

normalizer = Normalizer(inputCol="tfidf", outputCol="tfidf_norm", p=2.0)
df = normalizer.transform(df)
df = df.drop("tfidf")

print("✓ Vectores normalizados")
print("\nDataFrame final:")
df.select("book_id", "title", "tfidf_norm").show(4, truncate=60)

Normalizando vectores TF-IDF...
✓ Vectores normalizados

DataFrame final:
+---------------------------+-------------------------------+------------------------------------------------------------+
|                    book_id|                          title|                                                  tfidf_norm|
+---------------------------+-------------------------------+------------------------------------------------------------+
|If the Eternity Should Fail|If the Eternity Should Fail.txt|(93,[1,2,3,4,5,6,7,8,9,11,12,18,21,22,24,26,32,33,35,39,4...|
|            Man On The Edge|            Man On The Edge.txt|(93,[0,5,6,7,14,15,18,23,28,34,42,43,45,47,49,53,54,56,57...|
|             Speed Of Light|             Speed Of Light.txt|(93,[2,6,8,13,14,15,20,22,26,33,35,39,46,47,63,66,69,82,8...|
|          The Great Unknown|          The Great Unknown.txt|(93,[0,2,4,7,8,9,10,12,16,17,19,21,27,28,30,36,37,38,39,4...|
+---------------------------+-------------------------------+----

## CELDA 11: Crear Matriz de Similitud

**¿Qué hace?**
Crea una **matriz 100×100** que compara cada libro con todos los demás.

**Proceso:**
1. Convierte vectores de Spark a NumPy (arrays de Python)
2. Calcula el **producto punto** entre todos los pares de libros
3. El resultado es la **similitud coseno**


In [11]:
import numpy as np

print("Creando matriz de similitud...")

# Recolectar datos
rows = df.select("book_id", "title", "tfidf_norm").collect()

# Preparar estructuras de datos
book_ids = [r["book_id"] for r in rows]
id_to_title = {r["book_id"]: r["title"] for r in rows}

# 🆕 NUEVO: Crear índices numéricos
numeric_id_to_original = {i+1: book_ids[i] for i in range(len(book_ids))}  # 1, 2, 3...
original_to_numeric_id = {book_ids[i]: i+1 for i in range(len(book_ids))}

# Convertir vectores a numpy
print("  → Convirtiendo vectores a numpy...")
vectors = np.array([r["tfidf_norm"].toArray() for r in rows])

# Calcular matriz de similitud
print("  → Calculando similitudes...")
sim_matrix = np.dot(vectors, vectors.T)

# Crear mapeos (mantener los originales también)
index_to_id = {i: book_ids[i] for i in range(len(book_ids))}
id_to_index = {book_ids[i]: i for i in range(len(book_ids))}

print(f"✓ Matriz de similitud creada: {sim_matrix.shape}")
print(f"  Rango de similitudes: [{sim_matrix.min():.4f}, {sim_matrix.max():.4f}]")
print(f"  📊 {len(numeric_id_to_original)} libros indexados numéricamente")

Creando matriz de similitud...
  → Convirtiendo vectores a numpy...
  → Calculando similitudes...
✓ Matriz de similitud creada: (7, 7)
  Rango de similitudes: [0.0303, 1.0000]
  📊 7 libros indexados numéricamente


##  CELDA 12: Recomendar libros


**¿Qué hace?**\
Ingresamos un libro, seguido de ello se encuentra los N libros más similares (Por default y para evitar que la memoria muera pusimos 5).

**Pasos:**
1. Busca el libro en la matriz de similitud
2. Obtiene sus similitudes con todos los demás libros
3. Ordena de mayor a menor similitud
4. Retorna los top N


In [12]:
def recomendar_libros(libro_id, N=5):
    # Convertir a string para comparación
    libro_id_str = str(libro_id)
    
    # 🆕 Intentar con ID numérico primero
    if libro_id_str.isdigit():
        num_id = int(libro_id_str)
        if num_id in numeric_id_to_original:
            libro_id_str = numeric_id_to_original[num_id]
            print(f"📖 Usando ID numérico {num_id} → {libro_id_str}")
    
    # Validar que existe
    if libro_id_str not in id_to_index:
        raise ValueError(f"❌ Libro '{libro_id}' no encontrado")
    
    # Resto del código igual...
    idx = id_to_index[libro_id_str]
    similarities = sim_matrix[idx]

    pairs = [
        (index_to_id[i], float(similarities[i]))
        for i in range(len(similarities))
        if i != idx
    ]

    pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
    
    # 🆕 Agregar ID numérico a los resultados
    results = [
        (original_to_numeric_id[bid], bid, id_to_title[bid], score)
        for bid, score in pairs_sorted[:N]
    ]
    
    return results

## CELDA 13: Función Palabras Importantes (NORMALIZADA)


**¿Qué hace?**\
Encuentra las M palabras más características de un libro.

**Proceso:**
1. Obtiene el vector TF-IDF normalizado del libro
2. Ordena las palabras por su score TF-IDF
3. Retorna las top M palabras

**¿Por qué usa `tfidf_norm`?**
Los valores están entre 0 y 1, más fáciles de interpretar como porcentajes.

**Entrada:**
- `libro_id` = "84"
- `M` = 5

**Salida:**
```
elizabeth → 0.64 (64% de importancia)
feelings  → 0.15 (15% de importancia)
henry     → 0.12 (12% de importancia)
```

In [13]:
from pyspark.sql.functions import col

def palabras_importantes(documento_id, M=10):
    
    doc_id_str = str(documento_id)
    
    # 🆕 Convertir ID numérico a original
    if doc_id_str.isdigit():
        num_id = int(doc_id_str)
        if num_id in numeric_id_to_original:
            doc_id_str = numeric_id_to_original[num_id]
    
    # Buscar documento
    row = df.filter(col("book_id") == doc_id_str).select("tfidf_norm").collect()
    
    if not row:
        raise ValueError(f"❌ Documento '{documento_id}' no encontrado")
    
    # Resto del código igual...
    tfidf_vector = row[0]["tfidf_norm"]
    vocab = cv_model.vocabulary
    
    items = list(zip(tfidf_vector.indices, tfidf_vector.values))
    items_sorted = sorted(items, key=lambda x: x[1], reverse=True)
    
    top_words = [
        (vocab[idx], float(val))
        for idx, val in items_sorted[:M]
    ]
    
    return top_words


print("✓ Función palabras_importantes() definida (versión normalizada)")

✓ Función palabras_importantes() definida (versión normalizada)


## INPUTS

### CELDA 14: Catálogo Completo
Muestra los 100 libros disponibles con sus IDs.

In [14]:
print("\n" + "="*80)
print("📚 CATÁLOGO COMPLETO DE LIBROS")
print("="*80)
print(f"\nTotal: {len(book_ids)} libros\n")

for num_id, original_id in numeric_id_to_original.items():
    print(f"{num_id:3}. [ID: {num_id:2}] {id_to_title[original_id]:40} (Original: {original_id})")


📚 CATÁLOGO COMPLETO DE LIBROS

Total: 7 libros

  1. [ID:  1] If the Eternity Should Fail.txt          (Original: If the Eternity Should Fail)
  2. [ID:  2] Man On The Edge.txt                      (Original: Man On The Edge)
  3. [ID:  3] Speed Of Light.txt                       (Original: Speed Of Light)
  4. [ID:  4] The Great Unknown.txt                    (Original: The Great Unknown)
  5. [ID:  5] The Red And The Black.txt                (Original: The Red And The Black)
  6. [ID:  6] The book of souls.txt                    (Original: The book of souls)
  7. [ID:  7] When The River Runs Deep.txt             (Original: When The River Runs Deep)


### CELDA 15: Recomendador con INPUT
El usuario ingresa:
- ID del libro
- Cuántas recomendaciones quiere
→ Sistema devuelve libros similares

In [15]:
libro_id = input("\n📖 Ingresa el ID del libro (número o nombre): ").strip()
n_recs = int(input("🔢 ¿Cuántas recomendaciones?: ") or "5")

try:
    print(f"\n{'='*80}")
    print(f"📖 BUSCANDO LIBRO...")
    print(f"{'='*80}\n")
    
    recomendaciones = recomendar_libros(libro_id, N=n_recs)
    
    print(f"🎯 TOP {n_recs} RECOMENDACIONES:\n")
    for i, (num_id, orig_id, title, score) in enumerate(recomendaciones, 1):
        print(f"{i}. [ID: {num_id:2}] [{score:.4f}] {title}")
    
    print(f"\n✅ {n_recs} recomendaciones generadas")
except ValueError as e:
    print(str(e))


📖 Ingresa el ID del libro (número o nombre):  2
🔢 ¿Cuántas recomendaciones?:  5



📖 BUSCANDO LIBRO...

📖 Usando ID numérico 2 → Man On The Edge
🎯 TOP 5 RECOMENDACIONES:

1. [ID:  6] [0.1823] The book of souls.txt
2. [ID:  4] [0.1001] The Great Unknown.txt
3. [ID:  5] [0.0834] The Red And The Black.txt
4. [ID:  7] [0.0627] When The River Runs Deep.txt
5. [ID:  1] [0.0454] If the Eternity Should Fail.txt

✅ 5 recomendaciones generadas


### CELDA 16: Palabras Características con INPUT
El usuario ingresa:
- ID del libro
- Cuántas palabras quiere
→ Sistema muestra palabras clave con barras visuales

In [16]:
# CELDA 19: Palabras Características con INPUT (CORREGIDA)

libro_id = input("\n📖 Ingresa el ID del libro (número o nombre): ").strip()
m_palabras = int(input("🔢 ¿Cuántas palabras?: ") or "10")

try:
    # Convertir ID numérico si es necesario
    libro_id_str = str(libro_id)
    if libro_id_str.isdigit():
        num_id = int(libro_id_str)
        if num_id in numeric_id_to_original:
            libro_id_str = numeric_id_to_original[num_id]
            print(f"📖 Usando ID numérico {num_id} → {libro_id_str}")
    else:
        libro_id_str = libro_id
    
    # Validar que existe
    if libro_id_str not in id_to_title:
        print(f"❌ Libro '{libro_id}' no encontrado")
    else:
        print(f"\n{'='*80}")
        print(f"📖 LIBRO: {id_to_title[libro_id_str]}")
        print(f"{'='*80}\n")
        
        palabras = palabras_importantes(libro_id_str, M=m_palabras)
        
        for i, (palabra, score) in enumerate(palabras, 1):
            # Barra visual
            bar = "█" * int((score / palabras[0][1]) * 30)
            print(f"{i:2}. {palabra:20} │ {score:7.2f} │ {bar}")
        
        print(f"\n✅ Top {m_palabras} palabras generadas")
        
except Exception as e:
    print(f"❌ Error: {e}")


📖 Ingresa el ID del libro (número o nombre):  2
🔢 ¿Cuántas palabras?:  5


📖 Usando ID numérico 2 → Man On The Edge

📖 LIBRO: Man On The Edge.txt

 1. falling              │    0.93 │ ██████████████████████████████
 2. step                 │    0.11 │ ███
 3. wild                 │    0.11 │ ███
 4. around               │    0.11 │ ███
 5. head                 │    0.11 │ ███

✅ Top 5 palabras generadas
